## Setup

In [2]:
# Get raw advent-of-code data
from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=5)
input_data = puzzle.input_data
example = puzzle.examples[0]

In [3]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from common.utils.perf_check import check_time

## Part a

In [31]:
# Functions
def parse_input(input_data: str) -> tuple[list[int], list[list[list[int]]]]:
    """Parse the input data."""
    seeds_str, *map_strs = input_data.split("\n\n")
    seeds = [*map(int, seeds_str.split(":")[1].split())]
    maps = [[[*map(int, line.split())] for line in map_str.splitlines()[1:]] for map_str in map_strs]
    return seeds, maps


def map_seed(seed: int, maps: list[list[list[int]]]) -> int:
    """Map a single seed through all maps."""
    for m in maps:
        for dest, src, range_len in m:
            # Check if seed is in range
            if src <= seed < src + range_len:
                # If so, map it to the destination and continue to the next map
                seed += dest - src
                break
    return seed


def solve_a(input_data: str) -> int:
    """Solve part A of the puzzle."""
    seeds, maps = parse_input(input_data)
    return min(map_seed(seed, maps) for seed in seeds)

In [33]:
# Correctness check
assert str(solve_a(example.input_data)) == example.answer_a

In [173]:
# Performance check
time_a = check_time(solve_a, input_data)
print(f"Part A solution time: {time_a:.2f} ms")

Part A solution time: 0.18 ms


In [38]:
# Submit answer
puzzle.answer_a = solve_a(input_data)

## Part b

After checking the input data, it turns out the seed ranges contain more than 1.6 billion seeds. This makes it impossible to do the mapping for each number. Instead, we'll have to map the range starts and ends, and be clever about what happens if a seed range partially overlaps with a mapping range.

In [183]:
# Functions
def solve_b(input_data: str) -> int:
    """Solve part B of the puzzle."""
    seeds, maps = parse_input(input_data)

    # Convert seeds to (start, length) ranges
    ranges = list(zip(seeds[::2], seeds[1::2], strict=False))

    # Apply each map layer to all ranges
    for map_layer in maps:
        new_ranges = []

        for start, length in ranges:
            end = start + length
            # If a range overlaps with mapping rules, we may need to split it into sub-ranges
            # We need to check each yet unmapped sub-range separately
            sub_ranges = [(start, end)]

            # Try to map with each rule in this layer
            for dest, src, rng_len in map_layer:
                src_end = src + rng_len
                next_sub_ranges = []

                for r_start, r_end in sub_ranges:
                    # No overlap: passes through unchanged
                    if r_end <= src or r_start >= src_end:
                        next_sub_ranges.append((r_start, r_end))
                        continue

                    # Before overlap
                    if r_start < src:
                        next_sub_ranges.append((r_start, src))

                    # Overlapping part gets mapped
                    overlap_start = max(r_start, src)
                    overlap_end = min(r_end, src_end)
                    new_ranges.append((dest + (overlap_start - src), overlap_end - overlap_start))

                    # After overlap
                    if r_end > src_end:
                        next_sub_ranges.append((src_end, r_end))

                sub_ranges = next_sub_ranges

            # Add any unmapped ranges
            new_ranges.extend((s, e - s) for s, e in sub_ranges)

        # Update ranges for next layer
        ranges = new_ranges

    return min(start for start, _ in ranges)

In [184]:
# Correctness check
assert str(solve_b(example.input_data)) == example.answer_b

In [185]:
# Performance check
time_b = check_time(solve_b, input_data)
print(f"Part B solution time: {time_b:.2f} ms")

Part B solution time: 1.03 ms


In [186]:
# Submit answer
puzzle.answer_b = solve_b(input_data)

That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 5! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
